![workflow graph](Figures/SolutionNo_2_length_6.png "Workflow Graph")

In [1]:
from pathlib import Path
import sys

import pandas as pd

sys.path.append('/Users/stevep/Documents/code/APE_thesis/ape-thesis')
from wrapper_functions import *    

## Workflow Input Objects

### Table 1
- id: `titanic_train`
- source: `/Users/stevep/Documents/code/APE_thesis/ape-thesis/usecases/titanic/train.csv`
- DataClass: `MixedDataFrame`
- DataClass: `NoRelevance`    

In [2]:
titanic_train = load_table_csv('/Users/stevep/Documents/code/APE_thesis/ape-thesis/usecases/titanic/train.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


None

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Step 14: `column_split`
#### Notes
Splits a dataframe into X and y based on a column name
#### inputs:
- 1
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `NoRelevance`
	- APE_label: `['titanic_train']`
	- src: `(0, 9)`
- 2
	- DataClass: `IntColumn`
	- StatisticalRelevance: `DependentVariable`
	- APE_label: `['Survived']`
	- src: `(0, 0)`
#### outputs:
- 1
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
- 2
	- DataClass: `IntSeries`
	- StatisticalRelevance: `DependentVariable`

In [3]:
mixedDataFrame_14_1, intSeries_14_2 = column_split(df=titanic_train, column='Survived')

### Step 15: `train_test_split`
#### Notes
Splits a dataframe into X_train, y_train, X_test, y_test
    > returns strings instead of series if y is a string
    
#### inputs:
- 1
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
	- src: `(14, 0)`
- 2
	- DataClass: `IntSeries`
	- StatisticalRelevance: `DependentVariable`
	- src: `(14, 1)`
#### outputs:
- 1
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
- 2
	- DataClass: `IntSeries`
	- StatisticalRelevance: `DependentVariable`
- 3
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
- 4
	- DataClass: `IntSeries`
	- StatisticalRelevance: `DependentVariable`

In [4]:
mixedDataFrame_15_1, intSeries_15_2, mixedDataFrame_15_3, intSeries_15_4 = train_test_split(df=mixedDataFrame_14_1, y=intSeries_14_2)

### Step 16: `init_sklearn_voting_estimator`
#### Notes
Initializes a sklearn voting estimator.

    The passed voting estimator string must be one of the following:

    - 'VotingClassifier'
    - 'VotingRegressor'

    The passed estimator list string must be a comma seperated list of estimators
    (see `init_sklearn_estimator`)
    
#### inputs:
- 1
	- DataClass: `Str`
	- StatisticalRelevance: `BasicObjectRelevance_plain`
	- APE_label: `['RandomForestClassifier,LinearSVClassifier,LogisticRegressionClassifier']`
	- src: `(0, 10)`
#### outputs:
- 1
	- DataClass: `VotingClassifier`
	- StatisticalRelevance: `NoRelevance`

In [5]:
votingClassifier_16_1 = init_sklearn_voting_estimator(voting_estimator="VotingClassifier", estimator_list="RandomForestClassifier,LinearSVClassifier,LogisticRegressionClassifier")

### Step 17: `predict`
#### Notes
Predicts using a FITTED estimator
#### inputs:
- 1
	- DataClass: `VotingClassifier`
	- StatisticalRelevance: `NoRelevance`
	- src: `(16, 0)`
- 2
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
	- src: `(15, 0)`
#### outputs:
- 1
	- DataClass: `MixedSeries`
	- StatisticalRelevance: `Prediction`

In [6]:
mixedSeries_17_1 = predict(estimator=votingClassifier_16_1, X=mixedDataFrame_15_1)

NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### Step 18: `classification_report`
#### Notes
Displays a classification report
#### inputs:
- 1
	- DataClass: `MixedSeries`
	- StatisticalRelevance: `Prediction`
	- src: `(17, 0)`
- 2
	- DataClass: `IntSeries`
	- StatisticalRelevance: `DependentVariable`
	- src: `(15, 3)`
#### outputs:
- 1
	- DataClass: `ClassificationReport`
	- StatisticalRelevance: `NoRelevance`

In [ ]:
classificationReport_18_1 = classification_report(y_true=mixedSeries_17_1, y_pred=intSeries_15_4)

### Step 19: `fit_estimator`
#### Notes
Fits an estimator
    > Operation is in-place even though it returns the estimator!
    
#### inputs:
- 1
	- DataClass: `VotingClassifier`
	- StatisticalRelevance: `NoRelevance`
	- src: `(16, 0)`
- 2
	- DataClass: `MixedDataFrame`
	- StatisticalRelevance: `IndependentVariable`
	- src: `(15, 0)`
- 3
	- DataClass: `IntColumn`
	- StatisticalRelevance: `DependentVariable`
	- APE_label: `['Survived']`
	- src: `(0, 0)`
#### outputs:
- 1
	- DataClass: `VotingClassifier`
	- StatisticalRelevance: `NoRelevance`

In [ ]:
votingClassifier_19_1 = fit_estimator(estimator=votingClassifier_16_1, X=mixedDataFrame_15_1, y='Survived')